In [195]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2
from openai import AzureOpenAI  
from langchain.chat_models import AzureChatOpenAI


In [196]:
load_dotenv()

True

In [197]:
endpoint = os.getenv("ENDPOINT_URL")  
deployment = os.getenv("DEPLOYMENT_NAME")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")  

In [198]:

# Initialize Azure OpenAI LLM
llm = AzureChatOpenAI(
    openai_api_base=endpoint,  # Replace with your Azure OpenAI endpoint
    openai_api_version="2024-05-01-preview",  # Replace with your API version
    deployment_name=deployment,  # Replace with your deployment name
    openai_api_key=subscription_key,  # Replace with your Azure OpenAI API key
    openai_api_type="azure",
    model_name="gpt-4o" ,  # Replace with your model name
    temperature=0.3
)



c:\Users\HI\Documents\test-openai\projects\mcqgen\env\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://vijay-m6f1i5sd-eastus2.openai.azure.com/ to https://vijay-m6f1i5sd-eastus2.openai.azure.com/openai.
  warnings.warn(
c:\Users\HI\Documents\test-openai\projects\mcqgen\env\Lib\site-packages\langchain_community\chat_models\azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\HI\Documents\test-openai\projects\mcqgen\env\Lib\site-packages\langchain_community\chat_models\azure_openai.py:189: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias 

In [199]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
        }
    }

In [232]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
{response_json}
"""
#TEMPLATE

In [233]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )
#quiz_generation_prompt

In [234]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [235]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [236]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [237]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [238]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [239]:
file_path=r"C:\Users\HI\Documents\test-openai\projects\mcqgen\data.txt"
file_path

'C:\\Users\\HI\\Documents\\test-openai\\projects\\mcqgen\\data.txt'

In [240]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [241]:
print(TEXT)

Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]



In [242]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [243]:
NUMBER=1
SUBJECT="data science"
TONE="simple"

In [244]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 1 multiple choice questions for data science students in simple tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 1 MCQs
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english

In [246]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:679
Prompt Tokens:392
Completion Tokens:287
Total Cost:0.00385


In [254]:
response

{'text': 'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n',
 'number': 1,
 'subject': 'data science',
 'tone': 'simple',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'quiz': '```json\n{\n  "1": {\n    "mcq": "What does the field of data science primarily focus on?",\n    "options": {\n      "a": "Designing websites and applications",\n      "b": "Extracting knowledge and insights from data",\n      "c": "Building physical machines and robots",\n      "d": "Studying ancient civilizations"\n    },\n    "correct": "b"\n  }\n}\n```',
 'review': '### Complexity Analysis:\nThe question is straightforward and tests basic knowledge of data science. It 

In [260]:
qz=response.get('quiz')
print(qz)

```json
{
  "1": {
    "mcq": "What does the field of data science primarily focus on?",
    "options": {
      "a": "Designing websites and applications",
      "b": "Extracting knowledge and insights from data",
      "c": "Building physical machines and robots",
      "d": "Studying ancient civilizations"
    },
    "correct": "b"
  }
}
```
